In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import pymysql
from elasticsearch import Elasticsearch
import sys
sys.path.insert(0, '../')
import auth_prop

mysql_auth = auth_prop.mysql
# MySQL Connection 연결
con = pymysql.connect(host=mysql_auth['host'], user=mysql_auth['user'], password=mysql_auth['passwd']
    ,db=mysql_auth['db'], charset=mysql_auth['charset'])

# Connection 으로부터 Cursor 생성
cur = con.cursor()

# SQL문 실행 및 Fetch
sql = """SELECT NEWSITEMID 
FROM ABKL_NEWS_SUBJ_REFINE_1 ansr"""
cur.execute(sql)

# 데이타 Fetch
rows = cur.fetchall()
# 분류된 newsid 배열
id_arr = []
for id in rows:
    id_arr.append(id[0])

print(len(id_arr))

# ES 연결
es = Elasticsearch('http://localhost:9199')

# news_conts 담을 배열
conts_arr = []

In [ ]:
print(id_arr[46452])
id = id_arr[46452]

year = id.split(".")[1][:4]
index = 'kpf_bigkindslab_v1.1_'+year
body = {
    "_source": ["NewsItem.NewsComponent.NewsComponent.ContentItem.DataContent"],
    "query": {
        "match": {
            "_id": id
        }
    }
}
res = es.search(index=index, body=body)
print(res)

conts = res['hits']['hits'][0]
# ['_source']['NewsItem']['NewsComponent']['NewsComponent'][0]['ContentItem']['DataContent'].replace("&quot;","\"").replace("&apos;","'")
print(conts)

In [ ]:
# 조회할 es 정보
for idx, id in enumerate(id_arr):
    if idx!=0 and idx%100==0:
        print(idx)
        # DB에 학습데이터 insert
        for i,conts in enumerate(conts_arr):
            sql = """INSERT INTO ABKL_NEWS_SUBJ_LEARN_DATA 
                (NEWSITEMID, NEWS_CNTS, NEWS_BIG_SUBJ_CD, NEWS_SML_SUBJ_CD, NEWS_BIG_SUBJ_NM, NEWS_SML_SUBJ_NM)
                SELECT ansr.NEWSITEMID , %s AS NEWS_CNTS, ansr.NEWS_BIG_SUBJ_CD , ansr.NEWS_SML_SUBJ_CD , ansc.NEWS_BIG_SUBJ_NM , ansc.NEWS_SML_SUBJ_NM 
                FROM ABKL_NEWS_SUBJ_REFINE_1 ansr 
                    LEFT OUTER JOIN ABKL_NEWS_SUBJ_CD ansc ON ansr.NEWS_SML_SUBJ_CD = ansc.NEWS_SML_SUBJ_CD 
                WHERE NEWSITEMID = %s
                """
            cur.execute(sql, (conts, id_arr[idx-100+i]))
            con.commit()
            conts_arr = []
    
    year = id.split(".")[1][:4]
    index = 'kpf_bigkindslab_v1.1_'+year
    body = {
        "_source": ["NewsItem.NewsComponent.NewsComponent.ContentItem.DataContent"],
        "query": {
            "match": {
                "_id": id
            }
        }
    }

    res = es.search(index=index, body=body)
    conts = res['hits']['hits'][0]['_source']['NewsItem']['NewsComponent']['NewsComponent'][0]['ContentItem']['DataContent'].replace("&quot;","\"").replace("&apos;","'")
    conts_arr.append(conts)

# DB에 학습데이터 insert
# for i in range(len(conts_arr)):
#     sql = """INSERT INTO ABKL_NEWS_SUBJ_LEARN_DATA 
#         (NEWSITEMID, NEWS_CNTS, NEWS_BIG_SUBJ_CD, NEWS_SML_SUBJ_CD, NEWS_BIG_SUBJ_NM, NEWS_SML_SUBJ_NM)
#         SELECT ansr.NEWSITEMID , %s AS NEWS_CNTS, ansr.NEWS_BIG_SUBJ_CD , ansr.NEWS_SML_SUBJ_CD , ansc.NEWS_BIG_SUBJ_NM , ansc.NEWS_SML_SUBJ_NM 
#         FROM ABKL_NEWS_SUBJ_REFINE_1 ansr 
#             LEFT OUTER JOIN ABKL_NEWS_SUBJ_CD ansc ON ansr.NEWS_SML_SUBJ_CD = ansc.NEWS_SML_SUBJ_CD 
#         WHERE NEWSITEMID = %s
#         """
#     cur.execute(sql, (conts_arr[i], id_arr[i]))
#     con.commit()

In [ ]:
# DB connection 종료
con.close